# Student_Att
Give student's attendance percentage of each module per week 


## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as sql
import ipywidgets as widgets
from ipynb.fs.defs.CW_Preprocessing import *

## Functions 

In [2]:
def df_stu_with_sessions(df_student_attendance,dfsessions):
    '''
    merging student attendance and sessions dataframes
    '''
    
    df_student_attendance_T = df_student_attendance.transpose()
    df_student_attendance_T.index.names = ['session_id']
    df=pd.merge(dfsessions,df_student_attendance_T,\
                how='inner',on = 'session_id')
    return df

In [3]:
def calculate_att_per(df,studentId):
    '''
    creates a new column with attendance percentages 
    '''
    
    df_studentID = df[['course','week_no',studentId]]
    df_agg = df_studentID.groupby(['course','week_no']).size().reset_index(name='total_classes')
    df_attended = df_studentID.groupby(['course','week_no']).agg({studentId:'sum'}).reset_index()
    df_attended = df_attended.rename(columns={studentId:'classes_attended'})
    df_merged = pd.merge(df_agg, df_attended, on=['course','week_no'])
    df_merged['attendance_percentage'] = df_merged['classes_attended'] / df_merged['total_classes'] * 100
    
    return df_merged


In [4]:
def student_att(studentId):
    '''
    Shows attendance percentage for each module per week for the given student ID
    
    input:
        studentId: Student ID number, string type 
    return: 
        displays a table and a graph
    '''
    
    try:
        # Creating a attendance record table 
        df_student_attendance,dfsessions=get_tables()
        df = df_stu_with_sessions(df_student_attendance,dfsessions)
        df_merged = calculate_att_per(df,studentId)
        df_merged = df_merged.pivot(index='week_no',\
                                    columns='course', values='attendance_percentage')
        df_merged.columns.name = None
        df_merged = df_merged.rename(columns=\
                                     {'COA111':'COA111%','COA122':'COA122%'})

        # Creating an output widgets 
        table_out = widgets.Output()
        graph_out = widgets.Output()

        # Displaying the table and graph in the output widgets
        with table_out:
            display(df_merged)
        with graph_out:
            df_merged.plot.bar(figsize=(5,3))
            plt.xticks(rotation=0)
            plt.show()

        # Displaying the table and the graph in a horizontal box layout
        hbox = widgets.HBox([table_out, graph_out])
        display(hbox)
        
    except:
        print("Student ID: %s did not attended classes"%(studentId))
   

## Testing 

In [7]:
student_att(16)